# Day 9: Rope Bridge

In [1]:
from dataclasses import dataclass
from typing import List
import os

In [2]:
@dataclass(frozen=True)
class Point:
  x: int
  y: int

  def __repr__(self):
    return f'P({self.x}, {self.y})'

  def offset(self, relative: str):
    """ `relative` is a string like 'R 5' which gives a direction
    and a magnitude. The letter can be one of:
      Left: `L` for negative x
      Right: `R` for positive x
      Left: `U` for positive y
      Left: `D` for negative y
    """
    direction, magnitude = relative.split()
    unit = (0,0)
    match direction:
      case 'L':
        unit = (-1,0)
      case 'R':
        unit = (1,0)
      case 'U':
        unit = (0,1)
      case 'D':
        unit = (0,-1)
    magnitude = int(magnitude)
    return Point(self.x + unit[0] * magnitude, self.y + unit[1] * magnitude)

@dataclass
class Vector:
  start: Point
  end: Point

  def __repr__(self):
    return f'V({self.start}, {self.end})'

@dataclass
class Path:
  segments: List[Vector]

  #def add_segment(self, relative : str):
  #  self.segments.append

  def end_point(self) -> Point:
    if len(self.segments):
      return self.segments[-1].end
    return Point(0,0)

  def __repr__(self):
    return f'{self.segments}'

_Exercise classes_

In [3]:
def exercise_classes():
  path  = Path([])
  path_end = Point(0,0)
  for p in [Point(0, 0), Point(-5, 0), Point(0, 6), Point(10, 20)]:
    for q in ['L 1', 'R 1', 'U 1', 'D 1', 'L 5']:
      print(f'{p} + {q} = {p.offset(q)}, V = {Vector(p, p.offset(q))}')
      next_segment = Vector(path.end_point(), path.end_point().offset(q))
      path.segments.append(next_segment)
  print(f'Path {path}')

exercise_classes()

P(0, 0) + L 1 = P(-1, 0), V = V(P(0, 0), P(-1, 0))
P(0, 0) + R 1 = P(1, 0), V = V(P(0, 0), P(1, 0))
P(0, 0) + U 1 = P(0, 1), V = V(P(0, 0), P(0, 1))
P(0, 0) + D 1 = P(0, -1), V = V(P(0, 0), P(0, -1))
P(0, 0) + L 5 = P(-5, 0), V = V(P(0, 0), P(-5, 0))
P(-5, 0) + L 1 = P(-6, 0), V = V(P(-5, 0), P(-6, 0))
P(-5, 0) + R 1 = P(-4, 0), V = V(P(-5, 0), P(-4, 0))
P(-5, 0) + U 1 = P(-5, 1), V = V(P(-5, 0), P(-5, 1))
P(-5, 0) + D 1 = P(-5, -1), V = V(P(-5, 0), P(-5, -1))
P(-5, 0) + L 5 = P(-10, 0), V = V(P(-5, 0), P(-10, 0))
P(0, 6) + L 1 = P(-1, 6), V = V(P(0, 6), P(-1, 6))
P(0, 6) + R 1 = P(1, 6), V = V(P(0, 6), P(1, 6))
P(0, 6) + U 1 = P(0, 7), V = V(P(0, 6), P(0, 7))
P(0, 6) + D 1 = P(0, 5), V = V(P(0, 6), P(0, 5))
P(0, 6) + L 5 = P(-5, 6), V = V(P(0, 6), P(-5, 6))
P(10, 20) + L 1 = P(9, 20), V = V(P(10, 20), P(9, 20))
P(10, 20) + R 1 = P(11, 20), V = V(P(10, 20), P(11, 20))
P(10, 20) + U 1 = P(10, 21), V = V(P(10, 20), P(10, 21))
P(10, 20) + D 1 = P(10, 19), V = V(P(10, 20), P(10, 19))
P(10,

In [4]:
def load_data(filename : str) -> Path:
  p = Path([])
  with open(filename) as f:
    for line in f.readlines():
      line = line.strip()
      if not len(line) or line[0] == '#':
        continue
      e = p.end_point()
      p.segments.append(Vector(e, e.offset(line)))
  return p

In [5]:
def make_tail(head : Path) -> Path:
  """ Creates the next destination for `t` to approach `h` """
  def follow(h : Point, t : Point) -> Point:
    x : int = t.x
    y : int= t.y
    if h.x > t.x:
      x = h.x - 1
    if h.x < t.x:
      x = h.x + 1
    if h.y > t.y:
      y = h.y - 1
    if h.y < t.y:
      y = h.y + 1
    return Point(x, y)

  tail = Path([Vector(Point(0, 0), Point(0, 0))])
  for s in head.segments:
    tail_current = tail.end_point()
    head_current = s.end
    tail_next = follow(head_current, tail_current)
    tail.segments.append(Vector(tail_current, tail_next))
  return tail

In [6]:
def unique_positions(path : Path):
  # Find unique positions visited by the tail
  positions = set()
  for e in path.segments:
    positions.add(e.end)
  return positions

In [7]:
def exercise_unique_positions():
  for f in [os.path.join('testdata', t) for t in ['easy.txt', 'stay.txt', 'short.txt', 'sample.txt']]:
    h = load_data(f)
    t = make_tail(h)
    print(f'{f}: Path {t}, unique {unique_positions(t)}')  
  
exercise_unique_positions()

testdata/easy.txt: Path [V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 9)), V(P(0, 9), P(0, 19)), V(P(0, 19), P(9, 19)), V(P(9, 19), P(-9, 19))], unique {P(9, 19), P(0, 0), P(0, 19), P(0, 9), P(-9, 19)}
testdata/stay.txt: Path [V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0))], unique {P(0, 0)}
testdata/short.txt: Path [V(P(0, 0), P(0, 0)), V(P(0, 0), P(13, 0)), V(P(13, 0), P(13, -15)), V(P(13, -15), P(13, -15)), V(P(13, -15), P(23, -15))], unique {P(13, -15), P(13, 0), P(23, -15), P(0, 0)}
testdata/sample.txt: Path [V(P(0, 0), P(0, 0)), V(P(0, 0), P(13, 0)), V(P(13, 0), P(13, -15)), V(P(13, -15), P(13, -15)), V(P(13, -15), P(23, -15)), V(P(23, -15), P(23, -8)), V(P(23, -8), P(31, -8)), V(P(31, -8), P(31, -2)), V(P(31, -2), P(31, -7)), V(P(31, -7), P(31, -2))], unique {P(13, -15), P(31, -2), P(0, 0), P(13, 0), P(31, -7), P(23, -8), P(23, -15), P(31, -8)}


In [8]:
def solver():
  h = load_data('input.txt')
  t = make_tail(h)
  print(f'{len(unique_positions(t))}')  
  
solver()

1481
